In [1]:
import pickle
import pandas as pd
from pandas import Series
from tqdm import tqdm
import seaborn as sns
import numpy as np

import matplotlib
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import font_manager as fm
from matplotlib import rc


matplotlib.rcParams['figure.dpi'] = 150
f_path = "/System/Library/Fonts/Helvetica.ttc"
font_name = fm.FontProperties(fname=f_path).get_name()
rc('font', family=font_name, size=13)

## Finalize initial DDO Data preprocessing

In [2]:
df_original = pd.read_pickle('../dataset/02_ddo_initial_filtering/df_debater_vote_merged.p')

df_isbelief = pd.read_pickle('../dataset/03_belief_filtering/df_isbelief(gpt4_results).p')

In [3]:
df_original.head()

,debate_key,debate_title,username,debate_date,position
0,.-.-.-Ha-YOURE-GOING-DOWN-BEEM0R/1/,". . . Ha! YOU'RE GOING DOWN, BEEM0R!",Logical-Master,2009-02-02,Pro
1,.-Audis-are-junkers-except-to-rich-kids-with-l...,". Audis are junkers, except to rich kids with ...",Max.Wallace,2014-09-04,Pro
2,....-Former-Secretary-of-State-Madeleine-Albri...,"....""Former Secretary of State Madeleine Albri...",Lookingatissues,2017-01-30,Pro
3,...Words-can-t-hurt-me-any./1/,...Words can't hurt me any.,NonInDelicto,2007-12-19,Pro
4,.9-repeated-is-equal-to-1./1/,.9 repeated is equal to 1.,cowpie1998,2011-04-07,Pro


In [4]:
len(df_original)

232691

In [5]:
df_isbelief.head()

,debate_title,is_belief
0,". . . Ha! YOU'RE GOING DOWN, BEEM0R!",0
1,". Audis are junkers, except to rich kids with ...",1
2,"....""Former Secretary of State Madeleine Albri...",1
3,...Words can't hurt me any.,1
4,.9 repeated is equal to 1.,1


In [6]:
sum(df_isbelief['is_belief'] == 1), sum(df_isbelief['is_belief'] == 0), sum(df_isbelief['is_belief'] == 0)/len(df_isbelief)

(59986, 8904, 0.1292495282334156)

In [7]:
len(df_isbelief) #+ #no answer 10

68890

In [8]:
title2isbelief = dict(zip(df_isbelief['debate_title'], df_isbelief['is_belief']))
df = df_original.copy()
df['is_belief'] = df['debate_title'].map(title2isbelief)

In [9]:
df.head()

,debate_key,debate_title,username,debate_date,position,is_belief
0,.-.-.-Ha-YOURE-GOING-DOWN-BEEM0R/1/,". . . Ha! YOU'RE GOING DOWN, BEEM0R!",Logical-Master,2009-02-02,Pro,0.0
1,.-Audis-are-junkers-except-to-rich-kids-with-l...,". Audis are junkers, except to rich kids with ...",Max.Wallace,2014-09-04,Pro,1.0
2,....-Former-Secretary-of-State-Madeleine-Albri...,"....""Former Secretary of State Madeleine Albri...",Lookingatissues,2017-01-30,Pro,1.0
3,...Words-can-t-hurt-me-any./1/,...Words can't hurt me any.,NonInDelicto,2007-12-19,Pro,1.0
4,.9-repeated-is-equal-to-1./1/,.9 repeated is equal to 1.,cowpie1998,2011-04-07,Pro,1.0


In [10]:
df[df.is_belief==0]['debate_title'].sample(100).to_list()

['Affirmative Action',
 'Psychiatry',
 'Lynch all liars.',
 'Quotes',
 'Gay Marraige',
 'Abortion',
 'World War 2 Code Debate',
 '"Shipping" in My Little Pony fanfictions is ...',
 'My Pen is bigger than yours.',
 'Barack Obama was recently pwnt by Rio de Janeiro',
 'Hodor Hodor Hodor hodor hodor hodor',
 'Community Service',
 'Net neutrality laws.',
 'Halo Or Call of Duty.',
 'War on terror',
 'The BAD argument debate',
 'Longer Lunches',
 'This is not valid in content',
 'Rap battle 1',
 'Death penalty',
 'Insult war',
 'Bill Cipher w/a physical body vs Discord (Gravity Falls vs My Little Pony FiM)',
 'Abortion',
 'Assault rifles',
 'The Legalization of Prostitution',
 'Resolved: On balance, the United States federal govenment is justified in interveening in the intern',
 'Dinosaurs',
 'funny yo mama jokes',
 'Because we can.',
 'wanna be friends?',
 'Albert Wesker Vs',
 'Cannabis Legalization in the United States (Medical and Recreational Uses)',
 'Affirmative Action',
 'Short Story

In [13]:
df.to_pickle('../dataset/03_Final_dataframe/df_ddo_with_isbelief_info(N232691).p')

In [14]:
df_filtered = df[df['is_belief']==1]
df_filtered.head()
len(df_filtered)

197306

In [15]:
df_filtered.head()

,debate_key,debate_title,username,debate_date,position,is_belief
1,.-Audis-are-junkers-except-to-rich-kids-with-l...,". Audis are junkers, except to rich kids with ...",Max.Wallace,2014-09-04,Pro,1.0
2,....-Former-Secretary-of-State-Madeleine-Albri...,"....""Former Secretary of State Madeleine Albri...",Lookingatissues,2017-01-30,Pro,1.0
3,...Words-can-t-hurt-me-any./1/,...Words can't hurt me any.,NonInDelicto,2007-12-19,Pro,1.0
4,.9-repeated-is-equal-to-1./1/,.9 repeated is equal to 1.,cowpie1998,2011-04-07,Pro,1.0
5,.99-is-equal-to-one./1/,.99 is equal to one.,SweetCrackerJack,2013-12-24,Pro,1.0


In [16]:
#add phrase to make belief statement

pro_phrase = 'I agree with the following: '
con_phrase = 'I disagree with the following: '

phrases = []
for i in range(len(df_filtered)):
    position = df_filtered['position'].iloc[i]
    
    if position == 'Pro':
        phrases.append(pro_phrase + df_filtered['debate_title'].iloc[i])
    else:
        phrases.append(con_phrase + df_filtered['debate_title'].iloc[i])

In [17]:
len(phrases)

197306

In [18]:
df_filtered['belief_statement'] = phrases

/var/folders/r8/jdw1p2tj6jn34wf676kcsw_r0000gn/T/ipykernel_81853/3615872255.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['belief_statement'] = phrases


In [20]:
df_filtered.to_pickle('../dataset/03_Final_dataframe/df_ddo_including_only_truebeliefs(N197306).p')

In [21]:
len(df_filtered)

197306

### Remove duplicated data

In [29]:
len(df_filtered[df_filtered.duplicated()])

4999

In [30]:
df_filtered_nodup = df_filtered[~df_filtered.duplicated()]

In [31]:
len(df_filtered_nodup)

192307

In [32]:
df_filtered_nodup.to_pickle('../dataset/03_Final_dataframe/df_ddo_including_only_truebeliefs_nodup(N192307).p')